<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/iris_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
from torch import nn

import numpy as np

In [2]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn import datasets
import pandas as pd

# Load the Iris dataset
iris = datasets.load_iris()

# Create a DataFrame from the dataset
iris_df = pd.DataFrame(iris.data)
iris_df['class'] = iris.target
iris_df.columns = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']

In [5]:
df = iris_df.sample(frac = 1)
df_train,df_test = train_test_split(df,test_size = 0.1, random_state = 1)
y_train = df_train['class']
y_test = df_test['class']
X_train = df_train.drop('class', axis = 1)
X_test = df_test.drop('class', axis = 1)

In [6]:
y_train_t,y_test_t,X_train_t,X_test_t = torch.from_numpy(y_train.to_numpy()),torch.from_numpy(y_test.to_numpy()),torch.from_numpy(X_train.to_numpy()),torch.from_numpy(X_test.to_numpy())

In [7]:
y_train_t = y_train_t.type(torch.long).to(device)
y_test_t = y_test_t.type(torch.long).to(device)
X_train_t = X_train_t.type(torch.float32).to(device)
X_test_t = X_test_t.type(torch.float32).to(device)

In [8]:
class nn_model(nn.Module):
  def __init__(self):
    super().__init__()
    self.stack = nn.Sequential(
        nn.Linear(4,4),
        nn.ReLU(),
        nn.Linear(4,3)
    )

  def forward(self, X ):
    return self.stack(X)

In [9]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [10]:
model_0 = nn_model()

In [11]:
model_0.to(device)

nn_model(
  (stack): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=3, bias=True)
  )
)

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr=0.1)

In [13]:
torch.manual_seed(42)
epochs = 1000


In [14]:
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train_t) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train_t)
    acc = accuracy_fn(y_true=y_train_t,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_0(X_test_t)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test_t)
      test_acc = accuracy_fn(y_true=y_test_t,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 1.64213, Acc: 35.56% | Test Loss: 1.16072, Test Acc: 33.33%
Epoch: 10 | Loss: 0.84858, Acc: 79.26% | Test Loss: 0.80644, Test Acc: 86.67%
Epoch: 20 | Loss: 0.65002, Acc: 78.52% | Test Loss: 0.60191, Test Acc: 93.33%
Epoch: 30 | Loss: 0.53872, Acc: 91.85% | Test Loss: 0.49825, Test Acc: 100.00%
Epoch: 40 | Loss: 0.47493, Acc: 96.30% | Test Loss: 0.43961, Test Acc: 100.00%
Epoch: 50 | Loss: 0.42455, Acc: 97.04% | Test Loss: 0.39297, Test Acc: 100.00%
Epoch: 60 | Loss: 0.38113, Acc: 96.30% | Test Loss: 0.38292, Test Acc: 93.33%
Epoch: 70 | Loss: 0.55011, Acc: 64.44% | Test Loss: 0.75020, Test Acc: 46.67%
Epoch: 80 | Loss: 0.41688, Acc: 67.41% | Test Loss: 0.55959, Test Acc: 46.67%
Epoch: 90 | Loss: 0.42384, Acc: 67.41% | Test Loss: 0.57364, Test Acc: 46.67%
Epoch: 100 | Loss: 0.40977, Acc: 69.63% | Test Loss: 0.55436, Test Acc: 53.33%
Epoch: 110 | Loss: 0.40055, Acc: 72.59% | Test Loss: 0.54409, Test Acc: 53.33%
Epoch: 120 | Loss: 0.38977, Acc: 72.59% | Test Loss: 0.52946